## fake news classifer

### import libraries and load datasets

In [44]:
import pandas as pd
import numpy as np

import tensorflow as tf
from keras.layers import Embedding
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import one_hot
from keras.layers import LSTM, Dense, Dropout
from keras.models import Sequential

import nltk
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score

In [5]:
train = pd.read_csv('train.csv')
train.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [6]:
train = train.dropna()
x = train.drop('label',axis=1)
y = train['label']

In [8]:
print(x.shape)
print(y.shape)

(18285, 4)
(18285,)


### onehot representation

In [12]:
voc_size=5000
messages = x.copy()

In [13]:
messages.title[1]

'FLYNN: Hillary Clinton, Big Woman on Campus - Breitbart'

In [15]:
messages.reset_index(inplace=True)

In [21]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\J7000\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [23]:
#Datasets preprocessing
ps = PorterStemmer()
corpus = []
for i in range(0, len(messages)):
    review = re.sub('[^a-zA-Z]', ' ', messages['title'][i])
    review = review.lower()
    review = review.split()
    
    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

In [24]:
corpus

['hous dem aid even see comey letter jason chaffetz tweet',
 'flynn hillari clinton big woman campu breitbart',
 'truth might get fire',
 'civilian kill singl us airstrik identifi',
 'iranian woman jail fiction unpublish stori woman stone death adulteri',
 'jacki mason hollywood would love trump bomb north korea lack tran bathroom exclus video breitbart',
 'beno hamon win french socialist parti presidenti nomin new york time',
 'back channel plan ukrain russia courtesi trump associ new york time',
 'obama organ action partner soro link indivis disrupt trump agenda',
 'bbc comedi sketch real housew isi caus outrag',
 'russian research discov secret nazi militari base treasur hunter arctic photo',
 'us offici see link trump russia',
 'ye paid govern troll social media blog forum websit',
 'major leagu soccer argentin find home success new york time',
 'well fargo chief abruptli step new york time',
 'anonym donor pay million releas everyon arrest dakota access pipelin',
 'fbi close hilla

In [26]:
onehot_rep = [one_hot(words, voc_size) for words in corpus]
onehot_rep

[[700, 655, 1813, 832, 1436, 2551, 3730, 2726, 3505, 1746],
 [1380, 1867, 3753, 792, 957, 2482, 4936],
 [1766, 4076, 3409, 4259],
 [3769, 3133, 2688, 4037, 3919, 1838],
 [3357, 957, 1785, 3636, 1868, 4237, 957, 4280, 4301, 2645],
 [2691,
  4542,
  1303,
  4341,
  865,
  4271,
  2361,
  1698,
  1358,
  4109,
  3134,
  3784,
  3091,
  4441,
  4936],
 [563, 1473, 4507, 1367, 3844, 1843, 1931, 2511, 1129, 2780, 4553],
 [1934, 2774, 459, 3256, 2279, 1026, 4271, 1541, 1129, 2780, 4553],
 [2267, 632, 1171, 3995, 2140, 2818, 3040, 2151, 4271, 178],
 [19, 2428, 31, 1188, 4077, 4714, 1444, 1928],
 [842, 1673, 2719, 3370, 220, 1646, 4793, 246, 2676, 3705, 372],
 [4037, 2761, 1436, 2818, 4271, 2279],
 [517, 2176, 2437, 4617, 3994, 1304, 1599, 2615, 4354],
 [3556, 555, 4760, 2197, 3437, 2984, 4234, 1129, 2780, 4553],
 [1876, 4265, 1976, 232, 1740, 1129, 2780, 4553],
 [719, 4041, 3778, 1671, 3623, 3441, 1038, 478, 2604, 1794],
 [3266, 1035, 1867],
 [1507, 498, 268, 4938, 4271, 3530, 4036, 4936],
 [2

### Embedding representation

In [28]:
sent_length = 20
embedded_docs = pad_sequences(onehot_rep, padding='pre', maxlen=sent_length)
print(embedded_docs)

[[   0    0    0 ..., 2726 3505 1746]
 [   0    0    0 ...,  957 2482 4936]
 [   0    0    0 ..., 4076 3409 4259]
 ..., 
 [   0    0    0 ..., 1129 2780 4553]
 [   0    0    0 ..., 3637 4331 3540]
 [   0    0    0 ..., 1622  725 4607]]


In [29]:
embedded_docs[0]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,  700,
        655, 1813,  832, 1436, 2551, 3730, 2726, 3505, 1746])

In [32]:
len(embedded_docs), y.shape

(18285, (18285,))

In [33]:
x_final = np.array(embedded_docs)
y_final = np.array(y)

In [34]:
print(x_final.shape)
print(y_final.shape)

(18285, 20)
(18285,)


In [36]:
#split data
x_train, x_test, y_train, y_test = train_test_split(x_final, y_final, test_size=0.33, random_state=42)

### create model

In [30]:
embedding_vector_features = 40
model = Sequential()
model.add(Embedding(voc_size, embedding_vector_features, input_length=sent_length))
model.add(LSTM(100))
model.add(Dense(1, activation='sigmoid'))

In [31]:
model.compile(
    loss='binary_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 20, 40)            200000    
_________________________________________________________________
lstm_1 (LSTM)                (None, 100)               56400     
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 101       
Total params: 256,501
Trainable params: 256,501
Non-trainable params: 0
_________________________________________________________________


In [37]:
#train model
model.fit(
    x_train,
    y_train,
    validation_data = (x_test, y_test),
    epochs=10,
    batch_size=64
)

Train on 12250 samples, validate on 6035 samples
Epoch 1/10
12250/12250 [==============================] - 17s 1ms/step - loss: 0.3212 - acc: 0.8495 - val_loss: 0.2042 - val_acc: 0.9123
Epoch 2/10
12250/12250 [==============================] - 14s 1ms/step - loss: 0.1344 - acc: 0.9464 - val_loss: 0.2003 - val_acc: 0.9190
Epoch 3/10
12250/12250 [==============================] - 15s 1ms/step - loss: 0.0931 - acc: 0.9657 - val_loss: 0.2298 - val_acc: 0.9153
Epoch 4/10
12250/12250 [==============================] - 14s 1ms/step - loss: 0.0664 - acc: 0.9767 - val_loss: 0.2664 - val_acc: 0.9084
Epoch 5/10
12250/12250 [==============================] - 14s 1ms/step - loss: 0.0429 - acc: 0.9851 - val_loss: 0.2946 - val_acc: 0.9069
Epoch 6/10
12250/12250 [==============================] - 14s 1ms/step - loss: 0.0286 - acc: 0.9908 - val_loss: 0.3449 - val_acc: 0.9123
Epoch 7/10
12250/12250 [==============================] - 14s 1ms/step - loss: 0.0197 - acc: 0.9936 - val_loss: 0.3766 - val_acc:

In [40]:
#performance metrics and accuracy
y_pred = model.predict_classes(x_test)
confusion_matrix(y_test, y_pred)

array([[3086,  333],
       [ 194, 2422]], dtype=int64)

In [43]:
accuracy_score(y_test, y_pred)

0.91267605633802817

### adding dropout

In [45]:
embedding_vector_features = 40
model = Sequential()
model.add(Embedding(voc_size, embedding_vector_features, input_length=sent_length))
model.add(Dropout(0.3))
model.add(LSTM(100))
model.add(Dropout(0.3))
model.add(Dense(1, activation='sigmoid'))

model.compile(
    loss='binary_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 20, 40)            200000    
_________________________________________________________________
dropout_1 (Dropout)          (None, 20, 40)            0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 100)               56400     
_________________________________________________________________
dropout_2 (Dropout)          (None, 100)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 101       
Total params: 256,501
Trainable params: 256,501
Non-trainable params: 0
_________________________________________________________________


In [46]:
#train model
model.fit(
    x_train,
    y_train,
    validation_data = (x_test, y_test),
    epochs=10,
    batch_size=64
)

Train on 12250 samples, validate on 6035 samples
Epoch 1/10
12250/12250 [==============================] - 17s 1ms/step - loss: 0.3514 - acc: 0.8343 - val_loss: 0.2017 - val_acc: 0.9162
Epoch 2/10
12250/12250 [==============================] - 15s 1ms/step - loss: 0.1473 - acc: 0.9419 - val_loss: 0.1976 - val_acc: 0.9183
Epoch 3/10
12250/12250 [==============================] - 15s 1ms/step - loss: 0.1051 - acc: 0.9605 - val_loss: 0.2066 - val_acc: 0.9186
Epoch 4/10
12250/12250 [==============================] - 15s 1ms/step - loss: 0.0747 - acc: 0.9742 - val_loss: 0.2360 - val_acc: 0.9183
Epoch 5/10
12250/12250 [==============================] - 15s 1ms/step - loss: 0.0532 - acc: 0.9820 - val_loss: 0.3187 - val_acc: 0.9183
Epoch 6/10
12250/12250 [==============================] - 15s 1ms/step - loss: 0.0366 - acc: 0.9878 - val_loss: 0.3465 - val_acc: 0.9140
Epoch 7/10
12250/12250 [==============================] - 15s 1ms/step - loss: 0.0266 - acc: 0.9909 - val_loss: 0.3999 - val_acc:

In [47]:
#performance metrics
y_pred_dropout = model.predict_classes(x_test)
confusion_matrix(y_test, y_pred)

array([[3086,  333],
       [ 194, 2422]], dtype=int64)

In [48]:
accuracy_score(y_test,y_pred_dropout)

0.90903065451532727